In [4]:
from typing import List
from unidecode import unidecode

In [5]:
WORD_LENGTH = 5

# Carregar o Léxico
Precisamos de uma lista de palavras válidas para a língua portuguesa - chamamos isso de léxico. Vamos utilizar um produzido por um time de especialistas da USP, no projeto [Unitex](http://www.nilc.icmc.usp.br/nilc/projects/unitex-pb/web/dicionarios.html).

Esse léxico possui mais de 75.000 palavras encontradas em textos em português. 

> ⚠ Uma análise do arquivo irá mostrar vários verbetes que não seriam válidos no term.ooo, como palavras em inglês (Windows ou wheel), pois esse léxico foi extraído de textos reais onde verbetes em outras línguas também são encontrados. Futuramente, poderíamos fazer uma redução dessa base para apenas palavras usuais.

Note que carregamos apenas palavras com 5 caracteres, uma vez que esse tamanho é fixo no `term.ooo`.

In [6]:
def load_dictionary(filename: str, word_length: int):
    """
    Loads a lexicon from a file, where the each word is the first token on a line.
    Tokens are separated by commas.
    """
    with open(filename, 'r', encoding='UTF-8') as f:
        for line in f:
            line = line.split(',')[0]
            line = line.strip()

            if len(line) == word_length:
                yield line


Removemos diátricos (não são necessários no jogo). Usamos `set` para remover palavras repetidas que surgem após a remoção.

In [7]:

words = map(unidecode, load_dictionary('./DELAS_PB.dic', WORD_LENGTH))
words = set(words)


Também removemos do léxico palavras que já sabemos que não são aceitas no jogo.

In [8]:
forbidden = list(load_dictionary('forbidden_words.dic', WORD_LENGTH))

words = list(filter(lambda word: word not in forbidden, words))


# Classificando um chute

Assumindo que a probabilidade de escolher uma palavra aleatória do léxico pelo Termo é uniforme para todas as palavras, precisamos determinar qual seria a melhor escolha. 

O jogo funciona através de chutes de palavras, que fornecem pistas sobre as letras que compõe a palavra. Para cada letra da palavra, o jogo informa se a letra não está presente na palavra, se está presente em uma posição diferente da de nosso chute e se está exatamente na posição do chute. 

> Por exemplo, imagine um léxico de 5 palavras de tamanho 3, e um alfabeto de 3 letras: 
> 
> ``` python
> ['CBA', 'BBC', 'CAC', 'BCA', 'CAB']
> ```
> 
> Se a palavra escolhida for 'BBC' e chutássemos 'BCA', iamos ter uma informação:
> 
> 
> | B | C | A |
> |:-:|:-:|:-:|
> |🟢 |🟡| 🔴 |
> 
> Com essas pistas, podemos descartar todas as palavras do léxico que não começam com B:
> 
> ``` python
> ['BBC', 'BCA']
> ```
> 
> E descartar todas que tem A:
> ``` python
> ['BBC']
> ```
> Que nos leva a respota correta.

O ideal então é realizar um chute que remova o maior número de palavras. Em nosso exemplo foram eliminadas 4 palavras de 5. Esse seria o ideal, mas veja o seguinte, vamos fazer o mesmo chute, mas considerar que a palavra escolhida foi BBC:

> Léxico:
> 
> ``` python
> ['CBA', 'BBC', 'CAC', 'BCA', 'CAB']
> ```
>
> Chute:
> 
> | B | C | A |
> |:-:|:-:|:-:|
> |🟡 |🟡| 🔴 |
>
> Eliminamos todas as palavras que não tem B:
>
> ``` python
> ['CBA', 'BBC', 'BCA', 'CAB']
> ```
>
> E agora todas que tem B na primeira posição:
>
> ``` python
> ['CBA', 'CAB']
> ```
> 
> Agora eliminamos as que não tem C:
>
> ``` python
> ['CBA', 'CAB']
> ```
> 
> E, finalmente, as que tem C na segunda posição:
> 
> ``` python
> ['CBA', 'CAB']
> ```
>
> Eliminamos apenas 3 palavras!
>

A eficácia do "chute" varia não só com o léxico, mas também com base na palavra que foi escolhida pelo jogo. Então, na realidade, existe uma probabilidade de quantas palavras são eliminadas por um determinado chute, com base na palavra que foi escolhida.

Em teoria, poderíamos combinar o número de palavras eliminadas para cada chute com a probabilidade de escolha de cada palavra (1/N, onde N é o número de palavras do léxico). Mas para isso teríamos que calcular as palavras eliminadas para cada chute N vezes. Como o léxico começa com milhares de palavras, pode ser demorado fazer esse cálculo. Uma solução melhor é se basear nas próprias letras da palavra chutada. Ao invés de assumir uma palavra escolhida (das quais há N), podemos primeiro verificar a probablidade de acertar todas as letras, depois de acertar duas, depois uma, depois errar todas, depois acertar uma e uma posição, etc.

De forma genérica

![](https://latex.codecogs.com/svg.image?%5Csum_%7Bi=3%7D%5E%7B0%7D%5Csum_%7Bj=0%7D%5E%7B2%7D%20P(X=x,%20%5Calpha=i-j,%20%5Cbeta=j)%20)


Onde α são as letras na posição correta e ß são as letras que estão na palavra mas não na posição correta.

A probabilidade P de hipótese se obtém contando o número de palavras no léxico que obdece a disposição de letras inferida.

In [9]:
# Modelamos um resultade de um guess:
from enum import Enum

class GuessResult(Enum):
	"""The word does not contains the letter."""
	NOT_CONTAINS = 0
	"""The word contains the letter but no in the informed position."""
	CONTAINS = 1
	"""The word contains the letter and in the informed position."""
	CORRECT = 2


def guess_probability(lexicon: List[str], guess: str) -> float:
	"""
	Returns the probability of a word being a guess.
	"""
	pass


In [44]:
def filter_by_guess(lexicon: List[str], guess: str, guess_result: List[GuessResult]) -> List[str]:
    """
    Filter the lexicon by the guess result.
    """
    
    def is_match(guess: str, guess_result: List[GuessResult], word: str) -> bool:
        """
        Returns if the word matches the guess.
        """
        for i, guess_letter in enumerate(guess):
            if guess_result[i] == GuessResult.NOT_CONTAINS:
                if guess_letter in word:
                    return False
            elif guess_result[i] == GuessResult.CONTAINS:
                if guess_letter not in word or guess[i] == guess_letter:
                    return False
            elif guess_result[i] == GuessResult.CORRECT:
                if guess_letter != word[i]:
                    return False
            else:
                raise ValueError(f'Invalid guess result {guess_result[i]}')

        return True
    """
    Returns the lexicon filtered by the guess based on the result.
    """
    for word in lexicon:
        if is_match(guess, guess_result, word):
            yield word


assert list(filter_by_guess(['AB', 'AC', 'BA', 'BC', 'CA', 'CB'], 'BA', [GuessResult.CORRECT, GuessResult.NOT_CONTAINS])) == ['BC']
assert list(filter_by_guess(['AB', 'AC', 'BA', 'BC', 'CA', 'CB'], 'BA', [GuessResult.NOT_CONTAINS, GuessResult.CORRECT])) == ['CA']
assert list(filter_by_guess(['AB', 'AC', 'BA', 'BC', 'CA', 'CB'], 'BA', [GuessResult.CORRECT, GuessResult.CORRECT])) == ['BA']


# Primeiro chute

Agora, vamos encontrar a palavra para ser nosso melhor primeiro "chute". Considerando uma distribuição de palavras que sigam o léxico (algo que não é, necessariamente, verdade ☹), vamos verificar quais são as letras mais frequentes. Dessa forma, vamos fazer o primeiro chute com uma palavra que contenha as letras mais frequentes na língua.

Note que as cinco letras mais frequentes podem não formar uma palavra válida. Por isto, vamos pegar a palavra do léxico que possua o maior número de letras frequentes.

Primeiro, nós contabilizamos as letras no léxico.

In [10]:
letters = {}

for word in words:
    for letter in word:
        if letter in letters:
            letters[letter] += 1
        else:
            letters[letter] = 1

# Sort by most common letter
sorted_letters = sorted(letters.items(), key=lambda x: x[1], reverse=True)
sorted_letters = list(map(lambda x: x[0], sorted_letters))
'Ranked letters: ' + ', '.join(sorted_letters)

'Ranked letters: a, o, r, e, i, l, t, n, u, c, s, m, b, d, p, g, v, f, h, z, j, x, y, k, q, w'

Agora, procuramos uma palavra no léxico que maximize as letras frequentes. Um ponto importante aqui é que não queremos repetições - i.e., não queremos uma palavra que tenha letras repetidas.

In [11]:
def find_first_word_with_letters(words: List[str], letters: List[str]) -> str:
    """
    Finds the *first* word in the list that is made only of letters in the list.
    """
    for word in words:
        target = letters.copy()  # what letters are left to find

        for letter in word:
            if letter in target:
                target.remove(letter)

        if len(target) == len(letters) - len(word):
            return word

    return None


def find_next_best_guess(words, ranked_letters, guesses=[]):
    """
    Finds the best next guess based on the ranked letters.

    words - list of all possible words in the game
    ranked_letters - list of letters sorted by most common
    guesses - list of guesses made so far
    """

    # Build a list of all letters that already have been guessed
    guessed_letters = set([list(guess) for guess in guesses])               # list(str) split str by chars...

    # Filter the ranked letters to only include non-guessed letters
    if len(guessed_letters) != 0:
        print(f"Skipping letters: {guessed_letters}")        
        ranked_letters = filter(
            lambda letter: letter not in guessed_letters, ranked_letters)
        ranked_letters = list(ranked_letters)

    for size in range(WORD_LENGTH, len(ranked_letters)):
        letters = ranked_letters[:size]
        guess = find_first_word_with_letters(words, letters)

        if guess != None:
            return guess
        else:
            joined_letters = ''.join(letters)

            msg = f"Could not find a word using the top letters '{joined_letters}'. " #+ \
                #f"Only {len(ranked_letters)} letters are available."

            print(msg)


best_guess = find_next_best_guess(words, sorted_letters)
best_guess


Could not find a word using the top letters 'aorei'. 


'leira'

# Jogando o jogo

Com nosso primeiro chute em mãos, iniciamos o jogo. Precisamos de uma maneira para codificar a resposta que o jogo nos da para cada chute; utilizaremos uma string para isso, onde cada posição identifica o retorno do jogo, da seguinte maneira:

- Um espaço denota uma falha.
- Um caractere `i` indica que a letra está inclusa na palavra, mas não na posição correta.
- Um caracter `c` indica que a letra está inclusa na palavra e na posição correta.

Para jogar, informamos ao usuário qual deve ser o chute, e aguardamos que ele insira a resposta recebida. Aí computamos o melhor chute considerando o que já sabemos.

O melhor chute é aquele que obedece aquilo que já sabemos das palavras (por exemplo, quais letras devem estar presentes) e que prefere utilizar letras mais frequentes.

> Seguimos essa heurística por que não temos como deduzir a probabilidade de uma palavra aparecer no jogo, mas temos uma regra para determinar a probabilidade de uma palavra incluir uma letra. 

In [8]:
def validate_result(result: str) -> bool:
	if len(result) != 5:
		print('Result must be 5 letters long')
		return False
	
	if any(c not in ' ci' for c in result):
		print('Result must be composed of spaces, "i" or "c"')	
		return False

	return True

In [9]:
def describe_next_guess(included: List[str], not_included: List[str], known: List[str]) -> None:
    print('Guess must include {}'.format(', '.join(included)))
    print('Guess must not include {}'.format(', '.join(not_included)))
    print('Guess must be like {}'.format(', '.join(known)))


Para criar um novo chute, primeiro recuperamos a lista de palavras e a filtramos pelas regras.


In [10]:
def match(word: str, like: List[str], unlike: List[List[set]]) -> bool:
	for i in range(len(word)):
		if like[i] != ' ' and word[i] != like[i]:
			return False
		
		if word[i] in unlike[i]:
			return False

	return True


def filter_by(words: List[str], included_letters: List[str], not_included_letters: List[str], like: List[str], unlike: List[List[set]]):
	for word in words:
		# Check if any letter of the word is in the forbidden list
		if any(letter in not_included_letters for letter in word):
			continue
		# Check if any letter from the must-have list is missing
		if any(letter not in word for letter in included_letters):
			continue
		
		if not match(word, like, unlike):
			continue

		yield word		
	

In [11]:
def play_game(guesses: List[str], words: List[str], ranked_letters: List[str]):
    all_words = words.copy()

    # what letters should be in the word
    included_letters = set()

    # what letters should not be in the word
    not_included_letters = set()

    # the word that is being guessed, every letter is blank until it is known
    final_word = [letter for letter in '     ']

    # what letters must not be in each position in the word
    unlike = [set() for _ in range(5)]

    while True:
        print(f"Currently there are {len(words)} possible words.")

        result = input(
            f'Try the guess "{guesses[-1]}". What was the result? ').lower()

        if len(result) == 0:
            print("Empty result. Stopping game.")
            return

        if not validate_result(result):
            continue

        for i, r in enumerate(result):
            letter = guesses[-1][i]

            if r == ' ':
                not_included_letters.add(letter)
            elif r == 'c':
                final_word[i] = letter
            elif r == 'i':
                included_letters.add(letter)
                unlike[i].add(letter)

        #describe_next_guess(included_letters, not_included_letters, final_word)

        # update list of possible words based on the current state
        words = list(filter_by(words, included_letters,
                     not_included_letters, final_word, unlike))

        if len(words) == 0:
            print("Could not find any words that satisfy parameters. Stopping game.")
            break

        elif len(words) <= 10:
            print("Possible words: " + ', '.join(words))
            break

        else:
            # find the best guess
            guesses.append(find_best_guess(all_words, ranked_letters, guesses))


play_game([best_guess], words, sorted_letters)


Currently there are 3559 possible words.
Skipping letters: {'o', 'r', 'e', 'a', 'l'}
Could not find a word using the top letters 'itnuc'. 
Currently there are 121 possible words.
Skipping letters: {'o', 'r', 't', 'c', 'u', 'i', 's', 'e', 'a', 'l'}
Could not find a word using the top letters 'nmbdp'. 
Could not find a word using the top letters 'nmbdpg'. 
Could not find a word using the top letters 'nmbdpgv'. 
Could not find a word using the top letters 'nmbdpgvf'. 
Could not find a word using the top letters 'nmbdpgvfh'. 
Could not find a word using the top letters 'nmbdpgvfhz'. 
Could not find a word using the top letters 'nmbdpgvfhzj'. 
Could not find a word using the top letters 'nmbdpgvfhzjx'. 
Could not find a word using the top letters 'nmbdpgvfhzjxy'. 
Could not find a word using the top letters 'nmbdpgvfhzjxyk'. 
Could not find a word using the top letters 'nmbdpgvfhzjxykq'. 
Currently there are 12 possible words.
Empty result. Stopping game.
